# Cabinet Door Opening Robot - Interactive Notebook

This notebook walks you through the RoboCasa OpenCabinet task interactively.
Use this alongside the numbered Python scripts for a hands-on learning experience.

## Table of Contents
1. [Environment Setup](#1-environment-setup)
2. [Understanding the Task](#2-understanding-the-task)
3. [Observation and Action Spaces](#3-observation-and-action-spaces)
4. [Stepping Through the Environment](#4-stepping-through-the-environment)
5. [Visualizing Camera Views](#5-visualizing-camera-views)
6. [Exploring Kitchen Variety](#6-exploring-kitchen-variety)
7. [Dataset Inspection](#7-dataset-inspection)

## 1. Environment Setup

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Video

# Import RoboCasa (this registers all environments)
import robocasa
from robocasa.utils.env_utils import create_env

print(f"RoboCasa version: {robocasa.__version__}")
print(f"Available environments: {len(list(robocasa.ALL_KITCHEN_ENVIRONMENTS))}")

## 2. Understanding the Task

The **OpenCabinet** task requires a robot to:
1. Navigate near a closed kitchen cabinet
2. Reach for the cabinet door handle
3. Pull the door open

The task is complete when `fixture.is_open(env)` returns `True`.

In [ ]:
# Create the OpenCabinet environment
env = create_env(
    env_name="OpenCabinet",
    render_onscreen=False,
    seed=42,
    camera_widths=256,
    camera_heights=256,
)

obs = env.reset()
ep_meta = env.get_ep_meta()

print(f"Task: OpenCabinet")
print(f"Robot: {env.robots[0].name} ({env.robots[0].robot_model.__class__.__name__})")
print(f"Description: {ep_meta.get('lang', '')}")
print(f"Kitchen layout: {env.layout_id}, style: {env.style_id}")
print(f"Horizon: {env.horizon} steps at {env.control_freq} Hz = {env.horizon/env.control_freq:.0f}s")
print(f"Cabinet open? {env._check_success()}")

## 3. Observation and Action Spaces

In [ ]:
# List all observation keys
print("Observation keys:")
print(f"{'Key':<45} {'Shape':>15} {'Dtype':>10}")
print("-" * 75)
for key in sorted(obs.keys()):
    val = obs[key]
    if isinstance(val, np.ndarray):
        print(f"  {key:<43} {str(val.shape):>15} {str(val.dtype):>10}")

print(f"\nTotal observation keys: {len(obs)}")
print(f"Image observations: {sum(1 for k in obs if k.endswith('_image'))}")

In [ ]:
# Inspect the action space
robot = env.robots[0]
cc = robot.composite_controller

print(f"Action dimension: {env.action_dim}")
print(f"Controller: {cc.__class__.__name__}\n")

print("Action components:")
for part_name in cc.part_controllers:
    start, end = cc._action_split_indexes[part_name]
    print(f"  {part_name:<25} indices [{start}:{end}]  dim={end-start}")

## 4. Stepping Through the Environment

In [ ]:
# Run a few steps with zero (no-op) actions
obs = env.reset()

print("Running 10 steps with zero actions...")
for i in range(10):
    action = np.zeros(env.action_dim)
    obs, reward, done, info = env.step(action)
    success = env._check_success()
    print(f"  Step {i+1:2d}: reward={reward:.1f}, success={success}")

print("\nAs expected, doing nothing doesn't open the cabinet.")

In [ ]:
# Run a few steps with random actions
obs = env.reset()

print("Running 50 steps with random actions...")
rng = np.random.default_rng(42)
total_reward = 0
for i in range(50):
    action = rng.uniform(-1, 1, size=env.action_dim)
    # Zero out the base to prevent wild movement
    action[7:12] = 0.0  
    obs, reward, done, info = env.step(action)
    total_reward += reward

print(f"Total reward: {total_reward:.1f}")
print(f"Cabinet open? {env._check_success()}")
print("\nRandom arm movements don't reliably open cabinets either.")

## 5. Visualizing Camera Views

The PandaOmron has 3 cameras used for policy learning:
- **agentview_left**: Left shoulder camera
- **agentview_right**: Right shoulder camera  
- **eye_in_hand**: Wrist-mounted camera

In [ ]:
# Render camera views
obs = env.reset()

camera_names = [
    "robot0_agentview_left",
    "robot0_agentview_right",
    "robot0_eye_in_hand",
    "robot0_agentview_center",
]

fig, axes = plt.subplots(1, len(camera_names), figsize=(5 * len(camera_names), 5))
for ax, cam_name in zip(axes, camera_names):
    img = env.sim.render(height=256, width=256, camera_name=cam_name)[::-1]
    ax.imshow(img)
    short_name = cam_name.replace("robot0_", "")
    ax.set_title(short_name, fontsize=12)
    ax.axis("off")

plt.suptitle(f"Camera Views (layout={env.layout_id}, style={env.style_id})", fontsize=14)
plt.tight_layout()
plt.show()

## 6. Exploring Kitchen Variety

RoboCasa has 60+ kitchen layouts and 60+ styles. The policy must generalize
across all of them. Let's see a few different kitchens.

In [ ]:
# Show the same task in different kitchen scenes
scenes = [(11, 14), (15, 28), (18, 34), (40, 46), (50, 58)]

fig, axes = plt.subplots(1, len(scenes), figsize=(5 * len(scenes), 5))
for ax, (layout, style) in zip(axes, scenes):
    try:
        scene_env = create_env(
            env_name="OpenCabinet",
            render_onscreen=False,
            seed=42,
            camera_widths=256,
            camera_heights=256,
            layout_ids=layout,
            style_ids=style,
        )
        scene_env.reset()
        img = scene_env.sim.render(
            height=256, width=256, camera_name="robot0_agentview_center"
        )[::-1]
        ax.imshow(img)
        ax.set_title(f"L={layout}, S={style}", fontsize=11)
        scene_env.close()
    except Exception as e:
        ax.text(0.5, 0.5, f"Error\nL={layout}", ha="center", va="center", transform=ax.transAxes)
    ax.axis("off")

plt.suptitle("OpenCabinet in Different Kitchen Scenes", fontsize=14)
plt.tight_layout()
plt.show()

## 7. Dataset Inspection

After running `04_download_dataset.py`, you can inspect the demonstration data here.

In [ ]:
import os
from robocasa.utils.dataset_registry_utils import get_ds_path

dataset_path = get_ds_path("OpenCabinet", source="human")
print(f"Dataset path: {dataset_path}")

if dataset_path and os.path.exists(dataset_path):
    print("\nDataset structure:")
    for root, dirs, files in os.walk(dataset_path):
        level = root.replace(dataset_path, "").count(os.sep)
        if level > 2:
            continue
        indent = "  " * (level + 1)
        print(f"{indent}{os.path.basename(root)}/")
        for f in sorted(files)[:5]:
            print(f"{indent}  {f}")
        if len(files) > 5:
            print(f"{indent}  ... ({len(files)} files)")
else:
    print("\nDataset not downloaded yet. Run: python 04_download_dataset.py")

In [ ]:
# Try to load and inspect parquet data
if dataset_path and os.path.exists(dataset_path):
    try:
        import pyarrow.parquet as pq
        
        # Dataset path may point directly to the lerobot dir
        chunk_dir = os.path.join(dataset_path, "data", "chunk-000")
        if not os.path.exists(chunk_dir):
            chunk_dir = os.path.join(dataset_path, "lerobot", "data", "chunk-000")
        
        parquet_files = sorted(f for f in os.listdir(chunk_dir) if f.endswith(".parquet"))
        
        if parquet_files:
            table = pq.read_table(os.path.join(chunk_dir, parquet_files[0]))
            df = table.to_pandas()
            print(f"First parquet file: {parquet_files[0]}")
            print(f"Rows: {len(df)}, Columns: {len(df.columns)}")
            print(f"\nColumn names:")
            for col in df.columns:
                print(f"  {col}: dtype={df[col].dtype}")
            print(f"\nFirst 3 rows:")
            display(df.head(3))
    except ImportError:
        print("Install pyarrow to inspect parquet files: pip install pyarrow")
    except Exception as e:
        print(f"Could not load parquet data: {e}")

In [ ]:
# Clean up
env.close()
print("Environment closed. Notebook complete!")